In [1]:
import pandas as pd
import re

In [2]:
pd.options.display.max_columns = None

Merge Bursa, Antalya, and Ankara csv files

In [3]:
df_ankara = pd.read_csv(r"D:\repositories\repositories\data_analysis&science\HouseRentingPrices\hepsiEmlakAnkara.csv")
df_antalya = pd.read_csv(r"D:\repositories\repositories\data_analysis&science\HouseRentingPrices\hepsiEmlakAntalya.csv")
df_bursa = pd.read_csv(r"D:\repositories\repositories\data_analysis&science\HouseRentingPrices\hepsiEmlakBursa.csv")

In [4]:
df = pd.concat([df_ankara,df_bursa,df_antalya])

In [5]:
df = df.reset_index(drop=True)

Create columns for every distinct specs and distribute these specs to these columns

In [ ]:
# Split the each row of the specs with the "|??|" seperator (added when parsing the html)
# Then, rewrite it to the specs column
for x in df.index:
    df['specs'][x] = df['specs'][x].split("|??|")

In [7]:
# Create columns for each distinct specs
for x in df.index:
    for y in range(0, len(df['specs'][x])-1, 2):
        column_name = df['specs'][x][y]
        df[column_name] = ""

In [ ]:
# Fill the each distinct spec columns by using the "specs" column
for x in df.index:
    for y in range(0, len(df['specs'][x])-1, 2):
        df[df['specs'][x][y]][x] = df['specs'][x][y+1]

Clean the column names

In [9]:
df.columns = df.columns.str.replace(",","")
df.columns = df.columns.str.replace("'","")
df.columns = df.columns.str.replace("[","")
df.columns = df.columns.str.replace("]","")
df.columns = df.columns.str.strip()

Backup the df

In [10]:
df_backup = df.copy(deep=True)

General cleaning

In [11]:
for x in df.index:
    for y in df.columns:
        df.loc[x, f"{y}"] = re.sub(r"\[","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\]","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\\n","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\n","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub("\n","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\s\s","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\'","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = re.sub(r"\,","",str(df[f"{y}"][x]))
        df.loc[x, f"{y}"] = df[f"{y}"][x].strip()

Drop the unnecessary columns

In [12]:
df = df.drop(['title', 'specs', 'description', 'properties', 'İlan no', 'Son Güncelleme Tarihi', 'İlan Durumu',
       'Konut Tipi', 'Konut Şekli', 'Aidat', 'Depozito', 'Yakıt Tipi', 'Site İçerisinde',
       "Kira Getirisi", "Takas", "Parsel No","Krediye Uygunluk", "Ada No",
       "Cephe", "Tapu Durumu", "Kullanım Durumu", "Yapı Tipi"], axis='columns')

Turn prices to numbers

In [13]:
df['price'] = df['price'].str.replace(".","")
df['price'] = df['price'].str.replace(" TL","")
df['price'] = pd.to_numeric(df['price'])

Create the "rooms" columns and extract number of rooms from the "Oda + Salon Sayısı" column, then delete the 'Oda + Salon Sayısı'column

In [ ]:
df['rooms'] = ""
df['salon'] = ""

for x in df.index:
        df['Oda + Salon Sayısı'][x] = df['Oda + Salon Sayısı'][x].split(" + ")
        df['rooms'][x] = df['Oda + Salon Sayısı'][x][0]
        df['salon'][x] = df['Oda + Salon Sayısı'][x][1]

In [16]:
df = df.drop(['Oda + Salon Sayısı'], axis='columns')

Backup the df

In [19]:
df_backup2 = df.copy(deep=True)

Clean the 'Brüt / Net M2' column and create seperate columns for the Brüt and Net M2

In [36]:
df['Brüt / Net M2'] = df['Brüt / Net M2'].astype(str)

In [37]:
for x in df.index:
        df.loc[x, 'Brüt / Net M2'] = re.sub(r"\[","",str(df['Brüt / Net M2'][x]))
        df.loc[x, 'Brüt / Net M2'] = re.sub(r"\]","",str(df['Brüt / Net M2'][x]))
        df.loc[x, 'Brüt / Net M2'] = re.sub(r"\'","",str(df['Brüt / Net M2'][x]))
        df.loc[x, 'Brüt / Net M2'] = re.sub(r'\"',"",str(df['Brüt / Net M2'][x]))

In [43]:
for x in df.index:
        df.loc[x, 'Brüt / Net M2'] = re.sub(r"\s","",str(df['Brüt / Net M2'][x]))
        df.loc[x, 'Brüt / Net M2'] = re.sub(r"m2","",str(df['Brüt / Net M2'][x]))

In [ ]:
df['area'] = ""
df['net area'] = ""

for x in df.index:
        df['Brüt / Net M2'][x] = df['Brüt / Net M2'][x].split("/")
        df['area'][x] = df['Brüt / Net M2'][x][0]
        df['net area'][x] = df['Brüt / Net M2'][x][1]

In [48]:
df = df.drop(['Brüt / Net M2'], axis='columns')

Delete strings from the "Bina Yaşı" and change data type

In [50]:
for x in df.index:
        df.loc[x, 'Bina Yaşı'] = re.sub(r"\s","",str(df['Bina Yaşı'][x]))
        df.loc[x, 'Bina Yaşı'] = re.sub(r"Yaşında","",str(df['Bina Yaşı'][x]))

In [56]:
for x in df.index:
    if df.loc[x, 'Bina Yaşı'] == "SıfırBina":
        df.loc[x, 'Bina Yaşı'] = "0"

In [59]:
df["Bina Yaşı"] = pd.to_numeric(df["Bina Yaşı"])

Delete strings from the "Kat Sayısı	" and change data type

In [66]:
for x in df.index:
        df.loc[x, 'Kat Sayısı'] = re.sub(r"\s","",str(df['Kat Sayısı'][x]))
        df.loc[x, 'Kat Sayısı'] = re.sub(r"Katlı","",str(df['Kat Sayısı'][x]))

Turn some data to English

In [79]:
df.rename(columns={
    'Bulunduğu Kat': 'floor',
    'Bina Yaşı': 'age',
    "Isınma Tipi": "heating_type",
    "Kat Sayısı": "total_floors",
    "Eşya Durumu" : "furnished_house",
    "Banyo Sayısı" : "baths",
    }, inplace=True)

In [83]:
df = df.drop("Yapının Durumu", axis='columns')

In [100]:
for x in df.index:
    df.loc[x, 'floor'] = re.sub(r"Katı","floor",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Kat","floor",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Ara","Middle",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Zemin","Ground floor",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"En Üst","Last floor",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Yüksek Giriş","High entrance",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Kot 1","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Bahçe","Garden",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Giriş","Entrance",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Kot 3","-3",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Çatı","Roof",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Kot 2","-2",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Teras Giriş","Terrace",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Bodrum","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Yarı Bodrum","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Bodrum ve Zemin","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Villa","1.",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"-1 floor","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r" ve üzeri","-",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r"Yarı","-1",str(df['floor'][x]))
    df.loc[x, 'floor'] = re.sub(r" ve Ground floor","",str(df['floor'][x]))

Drop some outliers by cheking them with df['column_name'].unique() and sort_value and change data types

In [131]:
df = df.drop(df[df["price"]>1000000].index)

In [142]:
df = df.drop(df[df["floor"]==""].index)

In [151]:
df['age'] = df['age'].astype(int)
df = df.drop(df[df["age"]>99].index)

In [156]:
df = df.drop(df[df["total_floors"]==""].index)
df['total_floors'] = df['total_floors'].astype(int)
df = df.drop(df[df["total_floors"]>99].index)

In [161]:
df['rooms'] = df['rooms'].astype(int)
df = df.drop(df[df["rooms"]>9].index)

In [164]:
df['salon'] = df['salon'].astype(int)
df = df.drop(df[df["salon"]>9].index)

In [171]:
df = df.drop(df[df["area"]=="1.358"].index)
df = df.drop(df[df["area"]=="175.000"].index)
df = df.drop(df[df["area"]=="2.000"].index)
df = df.drop(df[df["area"]=="38.000"].index)
df = df.drop(df[df["area"]=="1.130"].index)

In [182]:
df['area'] = df['area'].astype(int)
df['net area'] = df['net area'].astype(int)

Save as csv file

In [184]:
df.to_csv(r"D:\repositories\repositories\data_analysis&science\HouseRentingPrices\cleaned_data.csv", index = False)